# 1. 텐서플로

- 강력한 수치 계산용 라이브러리
- 구글 브레인 팀에서 개발
- 구글 클라우드 스피치, 구글 포토, 구글 검색 대규모 서비스에 사용됨
- 2015년 11월 오픈 소스 공개
- GPU 지원
- 분산 컴퓨팅 지원
- Just-In-Time 컴파일러 포함 - 소도를 높이고 메모리 사용량을 즐이기 위해 계산을 최적화함
- 다른 환경에서 훈련과 실행이 가능하다
- 자동 미분 기능과 고성능 옵티마이저 RMSProp, Nadam 제공
- TensorFlow.js 자바스크립트 구현 있음, 브라우저에서 직접 모델 실행 가능한 것

저수준 텐서플로 연산 
- C++ 코드로 구현
- 여러 구현(kernel)이 존재 
- kernel은 CPU, GPU, TPU 같은 특정 장치에 맞춰 만들어짐

[TensorFlow GitHub](https://github.com/jtoy/awesome-tensorflow) 에서 많은 신경망 구조 다운로드 가능

TFX : TensorFlow 제품화를 위한 라이브러리 모음

[TensorFlow 버그 알리기, 새로운 기능 요청하기](https://github.com/tensorflow/tensorflow)    
[TensorFlow 커뮤니티](https://homl.info/41)

# 2. Using the TensorFlow LIKE NUMPY

##### tensor
- 한 연산에서 다른 연산으로 ... ~ tensorflow
- numpy - ndarray와 매우 비슷함
- 다차원 배열, 스칼라 값 가능    
텐서 생성과 조작

## 2.1 텐서와 연산
tf.constant()

1. 행렬

In [2]:
import tensorflow as tf

tf.constant([[1., 2., 3.], [4., 5., 6.]]) # matrix

2023-03-09 20:51:44.572241: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-09 20:52:39.707103: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

2. 스칼라

In [3]:
tf.constant(42) # scalar

<tf.Tensor: shape=(), dtype=int32, numpy=42>

3. 크기와 데이터 타입

In [4]:
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])
t, t.shape, t.dtype

(<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
 array([[1., 2., 3.],
        [4., 5., 6.]], dtype=float32)>,
 TensorShape([2, 3]),
 tf.float32)

## 2.2 Tensor and Numpy

넘파이 배열에 텐서플로 연산 가능    
텐서에 넘파이 연산 가능

## 2.3 타입 변환
타입 자동 변환 안함    
호환되지 않는 타입의 텐서로 연산시 (실수 텐서 * 정수 텐서) 예외 발생    
(32bit 실수 * 64bit 실수)    
변환 필요시 `tf.cast()` 함수 사용


## 2.4 변수
텐서의 내용은 수정 불가 = 역전파시 신경망의 가중치를 변경할 수 없음
-> tf.Variable 사용

In [5]:
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

- tf.Variable은 tf.Tensor와 비슷하게 작동함
- assign() 이나 scatter_update(), scatter_nd_update() 로 개별 원소를 수정 가능

In [6]:
v.assign(2 * v)
print(v)

v.scatter_nd_update(indices=[[0, 0], [1, 2]],
                    updates=[100., 200.])
print(v)

sparse_delta = tf.IndexedSlices(values=[[1., 2., 3.], [4., 5., 6.]],
                                indices=[1, 0])
v.scatter_update(sparse_delta)
print(v)

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>
<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[100.,   4.,   6.],
       [  8.,  10., 200.]], dtype=float32)>
<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[4., 5., 6.],
       [1., 2., 3.]], dtype=float32)>


## 2.5 데이터 구조
- 희소 텐서
- 텐서 배열 : 텐서의 리스트. 리스트에 포함된 모든 텐서 크기와 데이터 타입은 동일해야함
- 래그드 텐서 : 리스트의 리스트. 리스트의 길이는 다를 수 있음. tf.ragged package
- 문자열 텐서 : tf.string . 바이트 문자열을 나타냄(유니코드 아님!) 자동으로 UTF-8로 인코딩
- 집합 : 텐서 혹은 희소텐서로 표현. tf.sets package
- 큐 : tf.queue 

# 3. 사용자 정의 모델과 훈련 알고리즘

## 3.1 사용자 정의 손실 함수
train dataset noise -> 이상치 제거 혹은 데이터셋 수정 후에도 noise가 남아 있을거라고 가정    
이러한 경우 사용할 수 있는 손실 함수?

- 평균 제곱 오차 : 큰 오차에 큰 패널티가 주어지기 때문에 정확한 모델이 안만들어짐
- 평균 절댓값 오차 : 이상치에 관대, 수렴되는데 오래걸림
- 후버 손실 : L1과 L2의 장점과 단점을 보완, 모든 지점에서 미분이 가능

In [5]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = 1 * tf.abs(error) - 1 / 2
    return tf.where(is_small_error, squared_loss, linear_loss)

model.compile(loss=huber_fn, optimizer="nadam")
model.fit(X_train, y_train, ...)

In [1]:
import tensorflow as tf
huber_fn(0.98, 0.70)

2023-03-24 21:53:10.737342: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


NameError: name 'huber_fn' is not defined

## 3.2 Custom 요소를 넣은 Model Save & Load

1. Model Load

In [ ]:
model = keras.models.load_model("my_model.h5", 
                                custom_objects={"huber_fn":huber_fn})

상단의 후버 손실은 -1 ~ 1 사이의 오차는 작은것으로 간주됐다.    
만약 custom한 오차 함수에 다른 기준을 부여하려면 매개 변수를 받을 수 있는 함수를 만든다.    
threshold 값은 저장되지 않으므로, model load시 값을 지정해준다.

In [7]:
def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss  = threshold * tf.abs(error) - threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

In [8]:
model.compile(loss=create_huber(2.0), optimizer="nadam", metrics=["mae"])
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))
model.save("my_model_with_a_custom_loss_threshold_2.h5")

# 매개변수를 이용한 custom 함수 조정
model = keras.models.load_model("my_model_with_a_custom_loss_threshold_2.h5",
                                custom_objects={"huber_fn": create_huber(2.0)})

NameError: name 'model' is not defined

상단의 코드를 클래스 상속을 이용하여 구현할 수 있다.

In [ ]:
class HuberLoss(keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss  = self.threshold * tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}
    
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1),
])

model.compile(loss=HuberLoss(2.), optimizer="nadam", metrics=["mae"])

model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))
model.save("my_model_with_a_custom_loss_class.h5")


model = keras.models.load_model("my_model_with_a_custom_loss_class.h5",
                                custom_objects={"HuberLoss": HuberLoss})

In [ ]:
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

model.loss.threshold # thresshold값 확인 가능

## 3.3 활성화 함수, 초기화, 규제, 제한 Customizing

입, 출력을 가진 간단한 함수로 작성하면 된다.

In [ ]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

# 활성화 함수
def my_softplus(z): # return value is just tf.nn.softplus(z)
    return tf.math.log(tf.exp(z) + 1.0)

# 글로럿 초기화
def my_glorot_initializer(shape, dtype=tf.float32):
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)

# 사용자 정의 l1 규제
def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

# 양수인 가중치만 남기는 사용자 정의 제한
def my_positive_weights(weights): # return value is just tf.nn.relu(weights)
    return tf.where(weights < 0., tf.zeros_like(weights), weights)

# 레이어에 사용자 정의 함수들을 선언해준다
layer = keras.layers.Dense(1, activation=my_softplus,
                           kernel_initializer=my_glorot_initializer,
                           kernel_regularizer=my_l1_regularizer,
                           kernel_constraint=my_positive_weights)
    """
    활성화 함수 -> Dense 층 출력에 적용됨 -> 그 다음 층에 결과 전달
    가중치 : 초기화 함수 return값으로 초기화됨
    1st step -> 가중치가 규제 함수에 전달 -> 규제 손실 계산 -> 규제 손실값 전체 손실에 추가 
    -> 최종 손실 형성 -> 2nd step
    1st step -> 제한 함수 호출 -> 가중치를 제한한 가중치 값으로 변경 -> 2nd step
    """

keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1, activation=my_softplus,
                       kernel_regularizer=my_l1_regularizer,
                       kernel_constraint=my_positive_weights,
                       kernel_initializer=my_glorot_initializer),
])

# ... model fit ...생략
model.save("my_model_with_many_custom_parts.h5")

model = keras.models.load_model(
    "my_model_with_many_custom_parts.h5",
    custom_objects={
       "my_l1_regularizer": my_l1_regularizer,
       "my_positive_weights": my_positive_weights,
       "my_glorot_initializer": my_glorot_initializer,
       "my_softplus": my_softplus,
    })

함수가 모델과 함께 저장되어야 하는 하이퍼파라미터를 같이 가지고 있다면...    
손실, 활성화 함수 포함한 층 모델 - call() 메서드 있어야함    
규제, 초기화, 제한 모델은 - __call__() 메서드 있어야함

In [ ]:
class MyL1Regularizer(keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor
    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(self.factor * weights))
    def get_config(self):
        return {"factor": self.factor}

## 3.4 사용자 정의 지표

손실 :     
(cross entropy의 경우 ...) 훈련하려면 경사 하강법 사용 -> (평가할 영역) 미분 가능해야함 & gradient != 0 이어야 함 

지표 :     
(accuracy) 모델 평가 -> 미분 불가 or gradient != 0 이어도 ㄱㅊ  

Usually, 사용자 지표 함수 만드는 것 = 사용자 손실 함수 만드는 것

정밀도 = 예측 양성 / 찐 양성     
ex) 1st predict : 4 / 5 = 80%,     
2nd predict(잘못된 3개 양성 예측인 경우) : 0 / 5 = 0%     
1st and 2nd mean : 40% -> This is NOT REAL 정밀도

이처럼 배치마다 업데이트 되는 것 -> 스트리밍 지표

예측한 것 중 진짜 양성 (4 + 0) / 양성 예측 (5+3) = 50%
-> Precision

아래는 Precision 구현한 코드 keras 이용할 수 있음 

```
precision = keras.metrics.Pricision()
precision(y, y_hat)
```

In [ ]:
precision.result() # 현재 지표값 얻기
precision.variables # TP, FP 변수 확인 
precision.reset_states() # 위 변수 초기화

스트리밍 지표를 만들려면 `keras.metrics.Metric` 클래스를 상속하면 된다.    

아래는 전체 후버 손실 & 처리한 샘플 수 record - class 코드    
return 평균 후버 손실    
(아래 코드에서 keras가 지속적으로 변수를 관리하므로 따로 더 작성할 필요가 없는거다.)

In [ ]:
class HuberMetric(keras.metrics.Metric):
    def __init__(self, threshold=1.0, **kwargs):
        super().__init__(**kwargs) # handles base args (e.g., dtype)
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        # add_weight : 여러 지표 추가
        self.total = self.add_weight("total", initializer="zeros") # 후버 손실 합
        self.count = self.add_weight("count", initializer="zeros") # 처리한 총 샘플 수
    def update_state(self, y_true, y_pred, sample_weight=None): # 변수 업데이트, HuberMetric class를 함수로 사용할 때 호출 
        metric = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(metric))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))
    def result(self): # 최종 결과 계산 return
        return self.total / self.count
    def get_config(self): # threshold , model 저장
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

## 3.5 사용자 정의 층

특이한 층을 가진 네트워크 - 사용자 정의 층 

1. 가중치가 필요 없는 사용자 정의 층
keras.layers.Lambda 로 파이썬 함수 감싸기

(keras.layers.Flatten, keras.layers.RELU - 가중치가 없는 만들어져있는 층)

In [ ]:
exponential_layer = keras.layers.Lambda(lambda x: tf.exp(x))

exponential_layer를 사용할 곳은 여러군데다.    
[sequential API, 함수형 API, 서브클래싱 API, 활성화 함수]

2. 가중치가 필요한 사용자 정의 층 keras.layers.Layer 상속해야 함

